In [55]:
!pip install spotipy

In [56]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting up credientials; using with spotipy library

In [59]:
client_id = 'bfa31e8d61724c91a0ac3f96149f1016'
secret_key = 'bb257d6215e44fd5b859878c66f4aa89'

connection = SpotifyClientCredentials(client_id, client_secret = secret_key)
sp = spotipy.Spotify(client_credentials_manager = connection)

### Scraping 'Pop' Songs for their names only

In [60]:
sound_of_pop = '6gS3HhOiI17QNojjPuPzqc'

In [143]:
def get_playlist_tracks(playlist_genre):
    playlist = []
    for song in sp.playlist_tracks(playlist_genre)['items']:
        #get the uri for the song
        song_uri = song['track']['uri']
        #print(song)
        #get the name of the song
        song_name = song['track']['name']
        playlist.append(song_name)
    return playlist

In [107]:
# testing output of sound of pop to see if it loads the song names
get_playlist_tracks(sound_of_pop)

['Heart Attack',
 'One in a Million',
 'On My Love',
 'TiK ToK',
 'Love Me Like You Do - From "Fifty Shades Of Grey"',
 'Jealous - Remix',
 'Work from Home (feat. Ty Dolla $ign)',
 'Flowers',
 'Without Me',
 'Sucker',
 'Scars To Your Beautiful',
 'Made You Look',
 'Toxic',
 'Secret Love Song (feat. Jason Derulo)',
 '2002',
 'Let Me Go (with Alesso, Florida Georgia Line & watt)',
 'Praising You (feat. Fatboy Slim)',
 'Shameless',
 'Dark Horse',
 'Bang Bang',
 "There's Nothing Holdin' Me Back",
 'She Looks So Perfect',
 'Complicated',
 'Single Soon',
 'Dance The Night - From Barbie The Album',
 'I Don’t Wanna Live Forever (Fifty Shades Darker)',
 'Symphony (feat. Zara Larsson)',
 'Fancy',
 'Genie In a Bottle',
 'Attention',
 'Too Good At Goodbyes',
 'Since U Been Gone',
 'Unstoppable',
 'Somebody To You',
 'One Of Your Girls',
 'Whatcha Say',
 'This Town',
 'What A Time (feat. Niall Horan)',
 'Nonsense',
 'Cake By The Ocean',
 "I Ain't Worried",
 'On The Floor',
 'In The City',
 'Shallow

### Collecting Top 100 Genres from Everynoise

This is how each line of the Everynoise.com html source code looks like. Some preprocessing was done in excel to seperate out the tags with the playlist links and the names of the genres

In [63]:
""" <tr valign=top class=><td align=right class=note  style="font-size: 20px; line-height: 24px" >1</td><td  style="font-size: 20px; line-height: 24px"> <a href="https://embed.spotify.com/?uri=spotify:playlist:6gS3HhOiI17QNojjPuPzqc" class=note target=spotify title="See this playlist" onclick="linksync('https://embed.spotify.com/?uri=spotify:playlist:6gS3HhOiI17QNojjPuPzqc');">&#x260A;</a></td><td class=note  style="font-size: 20px; line-height: 24px"><a href="?root=pop&scope=all" title="Re-sort the list starting from here."  style="color: #AD8A07">pop</a></td></tr> """

' <tr valign=top class=><td align=right class=note  style="font-size: 20px; line-height: 24px" >1</td><td  style="font-size: 20px; line-height: 24px"> <a href="https://embed.spotify.com/?uri=spotify:playlist:6gS3HhOiI17QNojjPuPzqc" class=note target=spotify title="See this playlist" onclick="linksync(\'https://embed.spotify.com/?uri=spotify:playlist:6gS3HhOiI17QNojjPuPzqc\');">&#x260A;</a></td><td class=note  style="font-size: 20px; line-height: 24px"><a href="?root=pop&scope=all" title="Re-sort the list starting from here."  style="color: #AD8A07">pop</a></td></tr> '

In [64]:
topthousandgenres_uncleaned = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA340_NLP_FINAL/thousandtopsonggenres.csv')

In [65]:
topthousandgenres_uncleaned.head()

,PlaylistLink,Genre1,Genre2,Genre3,Genre4,Unnamed: 5,Unnamed: 6,GenreName
0,"href=""https://embed.spotify.com/?uri=spotify:p...","#AD8A07"">pop</a></td></tr>",NaN,NaN,NaN,"#AD8A07"">pop</a></td></tr>",NaN,"#AD8A07"">pop</a></td></tr>"
1,"href=""https://embed.spotify.com/?uri=spotify:p...","#A88A04"">rap</a></td></tr>",NaN,NaN,NaN,"#A88A04"">rap</a></td></tr>",NaN,"#A88A04"">rap</a></td></tr>"
2,"href=""https://embed.spotify.com/?uri=spotify:p...","#AC711A"">rock</a></td></tr>",NaN,NaN,NaN,"#AC711A"">rock</a></td></tr>",NaN,"#AC711A"">rock</a></td></tr>"
3,"href=""https://embed.spotify.com/?uri=spotify:p...","#BE9102"">urbano",latino</a></td></tr>,NaN,NaN,"#BE9102"">urbanolatino</a></td></tr>",NaN,"#BE9102"">urbanolatino</a></td></tr>"
4,"href=""https://embed.spotify.com/?uri=spotify:p...","#AD7F09"">hip",hop</a></td></tr>,NaN,NaN,"#AD7F09"">hiphop</a></td></tr>",NaN,"#AD7F09"">hiphop</a></td></tr>"


In [66]:
topthousandgenres_uncleaned_link = topthousandgenres_uncleaned[['PlaylistLink']]
topthousandgenres_uncleaned_link['PlaylistLink'] = topthousandgenres_uncleaned_link['PlaylistLink'].astype('str')
topthousandgenres_uncleaned_link.head(1)

,PlaylistLink
0,"href=""https://embed.spotify.com/?uri=spotify:p..."


### split and cleaned the tags with the playlist links to get only the link.

In [67]:
# I first tested on one url before creating a loop to do all of the urls
testurl = topthousandgenres_uncleaned_link['PlaylistLink'][0]
testurl

'href="https://embed.spotify.com/?uri=spotify:playlist:6gS3HhOiI17QNojjPuPzqc"'

In [68]:
# because of the quotations, the urls were outputting as an array and not a string so i had to join them
#testurl_joined = ''.join(testurl)
#testurl_joined

In [69]:
testurl_id = testurl.split(':',3)[3].split('"')[0]
testurl_id

'6gS3HhOiI17QNojjPuPzqc'

In [70]:
# now, the loop :)
ttg_playlist_id = pd.DataFrame(columns = ['PlaylistId'])

eachLink = 0

while eachLink < 1000:
    url = topthousandgenres_uncleaned_link['PlaylistLink'][eachLink]
    #url_joined = ''.join(testurl)
    url_id = url.split(':',3)[3].split('"')[0]
    #print(url_id)
    ttg_playlist_id = ttg_playlist_id.append({'PlaylistId':url_id}, ignore_index = True)

    eachLink = eachLink +1

ttg_playlist_id

,PlaylistId
0,6gS3HhOiI17QNojjPuPzqc
1,6s5MoZzR70Qef7x4bVxDO1
2,7dowgSWOmvdpwNkGFMUs6e
3,1kfJj9rUlOGawu2U1U4oZn
4,6MXkE0uYF4XwU4VTtyrpfP
...,...
995,1Wfx54pTW6GmYKD8IgnpTU
996,2JehmMfZgnbSe7e68JfElE
997,3pENYVTMuevP3CLjzQkDpQ
998,2gCb5sPRAaufQVrfBOLgdN


### parse, clean, and combine the columns with genre names and html to get one column with the complete genre name

In [71]:
topthousandgenres_uncleaned.head()

,PlaylistLink,Genre1,Genre2,Genre3,Genre4,Unnamed: 5,Unnamed: 6,GenreName
0,"href=""https://embed.spotify.com/?uri=spotify:p...","#AD8A07"">pop</a></td></tr>",NaN,NaN,NaN,"#AD8A07"">pop</a></td></tr>",NaN,"#AD8A07"">pop</a></td></tr>"
1,"href=""https://embed.spotify.com/?uri=spotify:p...","#A88A04"">rap</a></td></tr>",NaN,NaN,NaN,"#A88A04"">rap</a></td></tr>",NaN,"#A88A04"">rap</a></td></tr>"
2,"href=""https://embed.spotify.com/?uri=spotify:p...","#AC711A"">rock</a></td></tr>",NaN,NaN,NaN,"#AC711A"">rock</a></td></tr>",NaN,"#AC711A"">rock</a></td></tr>"
3,"href=""https://embed.spotify.com/?uri=spotify:p...","#BE9102"">urbano",latino</a></td></tr>,NaN,NaN,"#BE9102"">urbanolatino</a></td></tr>",NaN,"#BE9102"">urbanolatino</a></td></tr>"
4,"href=""https://embed.spotify.com/?uri=spotify:p...","#AD7F09"">hip",hop</a></td></tr>,NaN,NaN,"#AD7F09"">hiphop</a></td></tr>",NaN,"#AD7F09"">hiphop</a></td></tr>"


In [72]:
topthousandgenres_uncleaned_genre = topthousandgenres_uncleaned[['Genre1', 'Genre2', 'Genre3', 'Genre4']]
topthousandgenres_uncleaned_genre

,Genre1,Genre2,Genre3,Genre4
0,"#AD8A07"">pop</a></td></tr>",NaN,NaN,NaN
1,"#A88A04"">rap</a></td></tr>",NaN,NaN,NaN
2,"#AC711A"">rock</a></td></tr>",NaN,NaN,NaN
3,"#BE9102"">urbano",latino</a></td></tr>,NaN,NaN
4,"#AD7F09"">hip",hop</a></td></tr>,NaN,NaN
...,...,...,...,...
995,"#9F8C06"">rap",baiano</a></td></tr>,NaN,NaN
996,"#8F8707"">bhajan</a></td></tr>",NaN,NaN,NaN
997,"#AA880C"">hopebeat</a></td></tr>",NaN,NaN,NaN
998,"#A78A1A"">south",african,pop</a></td></tr>,NaN


## Some genres have up to four seperate words so I must accomodate for all the different ways that the html could be structured

### Case One: One word genre

In [73]:
testurl_case_one = topthousandgenres_uncleaned_genre['Genre1'][0]
testurl_case_one

'#AD8A07">pop</a></td></tr>'

In [74]:
testurl_genre_one = testurl_case_one.split('>')[1].split('<')[0]
testurl_genre_one

'pop'

### Case Two: Two word genre

In [75]:
testurl_case_two_1 = topthousandgenres_uncleaned_genre['Genre1'][3]
testurl_case_two_1

'#BE9102">urbano'

In [76]:
testurl_case_two_2 = topthousandgenres_uncleaned_genre['Genre2'][3]
testurl_case_two_2

'latino</a></td></tr>'

In [77]:
testurl_genre_two_1 = testurl_case_two_1.split('>')[1]
testurl_genre_two_1

'urbano'

In [78]:
testurl_genre_two_2 = testurl_case_two_2.split('<')[0]
testurl_genre_two_2

'latino'

In [79]:
testurl_genre_two = testurl_genre_two_1 + ' ' + testurl_genre_two_2
testurl_genre_two

'urbano latino'

### Case Three: three word genre

In [80]:
testurl_case_three_1 = topthousandgenres_uncleaned_genre['Genre1'][998]
testurl_case_three_1

'#A78A1A">south'

In [81]:
testurl_case_three_2 = topthousandgenres_uncleaned_genre['Genre2'][998]
testurl_case_three_2

'african'

In [82]:
testurl_case_three_3 = topthousandgenres_uncleaned_genre['Genre3'][998]
testurl_case_three_3

'pop</a></td></tr>'

In [83]:
testurl_genre_three_1 = testurl_case_three_1.split('>')[1]
testurl_genre_three_1

'south'

In [84]:
testurl_genre_three_2 = testurl_case_three_2
testurl_genre_three_2

'african'

In [85]:
testurl_genre_three_3 = testurl_case_three_3.split('<')[0]
testurl_genre_three_3

'pop'

In [86]:
testurl_genre_three = testurl_genre_three_1 + ' ' + testurl_genre_three_2 + ' ' + testurl_genre_three_3
testurl_genre_three

'south african pop'

### Case Four: Four word genre

In [87]:
testurl_case_four_1 = topthousandgenres_uncleaned_genre['Genre1'][97]
testurl_case_four_1

'#B4710E">east'

In [88]:
testurl_case_four_2 = topthousandgenres_uncleaned_genre['Genre2'][97]
testurl_case_four_2

'coast'

In [89]:
testurl_case_four_3 = topthousandgenres_uncleaned_genre['Genre3'][97]
testurl_case_four_3

'hip'

In [90]:
testurl_case_four_4 = topthousandgenres_uncleaned_genre['Genre4'][97]
testurl_case_four_4

'hop</a></td></tr>'

In [91]:
testurl_genre_four_1 = testurl_case_four_1.split('>')[1]
testurl_genre_four_1

'east'

In [92]:
testurl_genre_four_2 = testurl_case_four_2
testurl_genre_four_2

'coast'

In [93]:
testurl_genre_four_3 = testurl_case_four_3
testurl_genre_four_3

'hip'

In [94]:
testurl_genre_four_4 = testurl_case_four_4.split('<')[0]
testurl_genre_four_4

'hop'

In [95]:
testurl_genre_four = testurl_genre_four_1 + ' ' + testurl_genre_four_2 + ' ' + testurl_genre_four_3 + ' ' + testurl_genre_four_4
testurl_genre_four

'east coast hip hop'

### now, create a loop to handle all of the genres and their unique naming cases

In [96]:
ttg_playlist_genre = pd.DataFrame(columns = ['CompleteGenre'])
topthousandgenres_uncleaned_genre['Genre2'] = topthousandgenres_uncleaned_genre['Genre2'].astype('str')
topthousandgenres_uncleaned_genre['Genre3'] = topthousandgenres_uncleaned_genre['Genre3'].astype('str')
topthousandgenres_uncleaned_genre['Genre4'] = topthousandgenres_uncleaned_genre['Genre4'].astype('str')
topthousandgenres_uncleaned_genre.head()

,Genre1,Genre2,Genre3,Genre4
0,"#AD8A07"">pop</a></td></tr>",nan,nan,nan
1,"#A88A04"">rap</a></td></tr>",nan,nan,nan
2,"#AC711A"">rock</a></td></tr>",nan,nan,nan
3,"#BE9102"">urbano",latino</a></td></tr>,nan,nan
4,"#AD7F09"">hip",hop</a></td></tr>,nan,nan


In [97]:
eachLink = 0
while eachLink < 1000:

    if topthousandgenres_uncleaned_genre['Genre2'][eachLink] == 'nan' : #case one
        url_case_one = topthousandgenres_uncleaned_genre['Genre1'][eachLink]

        url_genre_one = url_case_one.split('>')[1].split('<')[0]

        ttg_playlist_genre = ttg_playlist_genre.append({'CompleteGenre':url_genre_one}, ignore_index = True)

    elif topthousandgenres_uncleaned_genre['Genre3'][eachLink] == 'nan' : #case two
        url_case_two_1 = topthousandgenres_uncleaned_genre['Genre1'][eachLink]
        url_case_two_2 = topthousandgenres_uncleaned_genre['Genre2'][eachLink]

        url_genre_two_1 = url_case_two_1.split('>')[1]
        url_genre_two_2 = url_case_two_2.split('<')[0]

        url_genre_two = url_genre_two_1 + ' ' + url_genre_two_2
        ttg_playlist_genre = ttg_playlist_genre.append({'CompleteGenre':url_genre_two}, ignore_index = True)

    elif topthousandgenres_uncleaned_genre['Genre4'][eachLink] == 'nan' : #case three
        url_case_three_1 = topthousandgenres_uncleaned_genre['Genre1'][eachLink]
        url_case_three_2 = topthousandgenres_uncleaned_genre['Genre2'][eachLink]
        url_case_three_3 = topthousandgenres_uncleaned_genre['Genre3'][eachLink]

        url_genre_three_1 = url_case_three_1.split('>')[1]
        url_genre_three_2 = url_case_three_2
        url_genre_three_3 = url_case_three_3.split('<')[0]

        url_genre_three = url_genre_three_1 + ' ' + url_genre_three_2 + ' ' + url_genre_three_3
        ttg_playlist_genre = ttg_playlist_genre.append({'CompleteGenre':url_genre_three}, ignore_index = True)

    else: #case four
        url_case_four_1 = topthousandgenres_uncleaned_genre['Genre1'][eachLink]
        url_case_four_2 = topthousandgenres_uncleaned_genre['Genre2'][eachLink]
        url_case_four_3 = topthousandgenres_uncleaned_genre['Genre3'][eachLink]
        url_case_four_4 = topthousandgenres_uncleaned_genre['Genre4'][eachLink]

        url_genre_four_1 = url_case_four_1.split('>')[1]
        url_genre_four_2 = url_case_four_2
        url_genre_four_3 = url_case_four_3
        url_genre_four_4 = url_case_four_4.split('<')[0]

        url_genre_four = url_genre_four_1 + ' ' + url_genre_four_2 + ' ' + url_genre_four_3 + ' ' + url_genre_four_4
        ttg_playlist_genre = ttg_playlist_genre.append({'CompleteGenre':url_genre_four}, ignore_index = True)


    eachLink = eachLink +1

ttg_playlist_genre

,CompleteGenre
0,pop
1,rap
2,rock
3,urbano latino
4,hip hop
...,...
995,rap baiano
996,bhajan
997,hopebeat
998,south african pop


### combining both dataframes so that the playlist link and playlist title are in the same dataframe!

In [98]:
ttg_playlist_genre = ttg_playlist_genre.reset_index()
ttg_playlist_id = ttg_playlist_id.reset_index()

In [99]:
complete_genre_df = pd.merge(ttg_playlist_genre, ttg_playlist_id, on='index', how='right')

In [100]:
complete_genre_df

,index,CompleteGenre,PlaylistId
0,0,pop,6gS3HhOiI17QNojjPuPzqc
1,1,rap,6s5MoZzR70Qef7x4bVxDO1
2,2,rock,7dowgSWOmvdpwNkGFMUs6e
3,3,urbano latino,1kfJj9rUlOGawu2U1U4oZn
4,4,hip hop,6MXkE0uYF4XwU4VTtyrpfP
...,...,...,...
995,995,rap baiano,1Wfx54pTW6GmYKD8IgnpTU
996,996,bhajan,2JehmMfZgnbSe7e68JfElE
997,997,hopebeat,3pENYVTMuevP3CLjzQkDpQ
998,998,south african pop,2gCb5sPRAaufQVrfBOLgdN


In [109]:
new_df = pd.DataFrame(get_playlist_tracks(complete_genre_df['PlaylistId'][97]))
new_df.columns = ['Songs']
new_df.head(2)

,Songs
0,Fast Life (with Nas)
1,I Like It (I Wanna Be Where You Are)


In [149]:
songs = pd.DataFrame()

In [152]:
for each_genre in range(1000):
  try:
    data_frame_name = complete_genre_df['CompleteGenre'][each_genre] + ' SONGS'
    playlist_id = complete_genre_df['PlaylistId'][each_genre]

    # Check if get_playlist_tracks returns a valid result
    playlist_tracks = get_playlist_tracks(playlist_id)
    if playlist_tracks is not None:
      songs[data_frame_name] = playlist_tracks
      print(each_genre)
  except Exception as e:
    print(f"Error processing genre {each_genre}: {e}")
    continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
Error processing genre 137: 'NoneType' object is not subscriptable
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
Error processing genre 195: 'NoneType' object is not subscriptable
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245


In [153]:
songs

,pop SONGS,rap SONGS,rock SONGS,urbano latino SONGS,hip hop SONGS,trap latino SONGS,dance pop SONGS,reggaeton SONGS,pop rap SONGS,modern rock SONGS,...,uk americana SONGS,tierra caliente SONGS,new age SONGS,instrumental rock SONGS,medieval rock SONGS,rap baiano SONGS,bhajan SONGS,hopebeat SONGS,south african pop SONGS,bubblegum dance SONGS
0,Heart Attack,Wake Up in the Sky,Paranoid - 2012 - Remaster,Lollipop,N.Y. State of Mind,Lollipop,Toxic,More,Kiss Me Thru The Phone,Cigarette Daydreams,...,Above The Clouds Of Pompeii,La Ultima Copa,Great Challenge,Chrome Castles,Spiel des Lebens,BALAH IH FOGOH,Shree RamcHandra Krupalu,Tear Us Apart,Suited,Dub-I-Dub
1,One in a Million,Praise The Lord (Da Shine) (feat. Skepta),Immigrant Song - Remaster,Holanda,C.R.E.A.M. (Cash Rules Everything Around Me) (...,Toda - Remix,Glamorous,Me Niegas,Nothin' on You (feat. Bruno Mars),Mind Over Matter (Reprise),...,Four Seasons in One Day,Borracho Te Recuerdo,Pachelbel's Canon in D,Tempest Pulse,Die hässlichen Kinder,Pretx Chave,Kabhi Ram Banke Kabhi Sham Banke,Skyline,Mthande,Superstar
2,On My Love,Going Bad (feat. Drake),Seven Nation Army,LALA,Ms. Fat Booty,"0 Sentimientos (Remix) [feat. Noriel, Darkiel,...",Genie In a Bottle,Ella Se Contradice (feat. Plan B),Buy U a Drank (Shawty Snappin') (feat. Yung Joc),First,...,Devil,No Somos Nada,Lady of the Moon,The Maze,Der Teufel...,Mundo Moderno,"Mangal Murti Maruti Nandan, Jai Jai Bajrang Bali",Dreaming,Collide,Superhero
3,TiK ToK,Love Me,Everlong,Toda - Remix,Don't Sweat The Technique,Netflixxx,The Sweet Escape,Sensacion Del Bloque,Dilemma,A-Punk,...,I'm on Fire,Aléjate,Walking Through Clouds,Heartburn,Brüder im Geiste,Não Sabe Quem Sou,Damodar Stuti,Eastside,Unspoken Word - Intro,Witch Doctor - Radio Mix
4,"Love Me Like You Do - From ""Fifty Shades Of Grey""",No Role Modelz,Baba O'Riley,Yandel 150,Can I Kick It?,Na' Personal,Don't Cha,Mi Cama Huele a Ti,My Chick Bad,Pumped Up Kicks,...,Wake Up Sleeper - Acoustic,Modesta Ayala,Entering Twin Falls,Beautiful Imperfection,Püppchen,Foice Facão e Peixera,Jai Hanuman Gyan,You're the One for Me - Digital Farm Animals R...,Imimoya,Macahula Dance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Say My Name,Into You (feat. Tamia) - Main Mix,Freak On a Leash,Un Cigarrillo,Always On Time,PLEBADA,The Boy Is Mine,Toma Toma,It Wasn't Me,Feel Good Inc.,...,Love Is...,El Baile de la Iguana (En Vivo),Son Of The Light,Sad & Illegal,Barrels of Whiskey,Necomancia,Santoshi Mata Ki Aarti,Skull and Bones,Cela Skhulume,Space Cowboy - Radio Edit
96,greedy,Dior,Wind Of Change,Nostálgico,1-800-273-8255,Me Rehúso,Try,Holanda,No Idea,Dog Days Are Over,...,Silver Linings,Monedita de Oro,Night Whispers,Omens And Portents 1: The Driver,I Hope They Sell Beer in Hell,Cidade Loka,radhe krishna dhuni,Digging up Skeletons,Umona (feat. Mpumi),"Gimme, Gimme, Gimme - Short Version"
97,Promiscuous,Real Muthaphuckkin G's,Owner of a Lonely Heart,TU$$I (with Dei V),Jump,Me Enteré,What About Us,Gasolina,Solid (feat. Drake),Sweet Disaster,...,One Last Trip Around The Sun,Un Angel no debe llorar,Moon Night,Power Symmetry,Wir sind des Geyers schwarzer Haufen,Perto (Outra Mentira),Dasanagu Visheshanagu,Move,Push Me to the Floor,Run Run Away
98,8 Letters,In Da Club,Drive,Serotonina,"Or Nah (feat. The Weeknd, Wiz Khalifa & DJ Mus...",JETSKI - Remix,Let Me Love You,No Debiste Volver,B.E.D.,Take on the World,...,Think I'm Alright Now,Me Gusta la Vida Recia,She Describes Infinity,Half Half & Half,Seemansgarnknäuel,Vou na Fé,Jay Aadhya Shakti,"June, After Dark",Tot Die Son Uitkom,More Than I Needed to Know
